In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

from sklearn.neighbors import KNeighborsClassifier

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

In [2]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup.csv")
X_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\scaled_validation_feature.csv")
y_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\validation_label.csv")

In [3]:
X_train = X_train.drop(["predict_cluster"],axis=1)
y_train = y_train.drop(["predict_cluster"],axis=1)
X_val = X_val.drop(["Unnamed: 0"],axis=1)
y_val = y_val.drop(["Unnamed: 0"],axis=1)

In [59]:
#select some of the data randomly
train = {'feature0':X_train['feature0'],'feature1':X_train['feature1'],'feature2':X_train['feature2'],'feature3':X_train['feature3'],'feature4':X_train['feature4'],'label':y_train['label']}
train = pd.DataFrame(data=train)
Class1_sample =pd.DataFrame.sample(train[train['label']==1],80000)
Class2_sample =pd.DataFrame.sample(train[train['label']==2],40000)
res = [Class1_sample, Class2_sample]
train_com = pd.concat(res)
sample_label = pd.DataFrame(train_com['label'])
sample_feature=train_com.drop(["label"],axis=1)

In [63]:
#double the data of class label 2
train = {'feature0':X_train['feature0'],'feature1':X_train['feature1'],'feature2':X_train['feature2'],'feature3':X_train['feature3'],'feature4':X_train['feature4'],'label':y_train['label']}
train = pd.DataFrame(data=train)
Class2_data =train[train['label']==2]
train = [train,Class2_data]
train = pd.concat(train)
train_double_label = pd.DataFrame(train['label'])
train_double_feature = train.drop(["label"],axis=1)

In [55]:
def cal_score (y_pred,y_val):
    n11 = 0
    n12 = 0
    n21 = 0
    n22 = 0
    y_pred_array= np.array(y_pred)
    y_val_array= np.array(y_val)
    for j in range(len(y_pred_array)):
        if (y_pred_array[j]==2)&(y_val_array[j]==2):
            n22 = n22+1
        elif (y_pred_array[j]==1)&(y_val_array[j]==2):
            n12 = n12 +1
        elif (y_pred_array[j]==2)&(y_val_array[j]==1):
            n21 = n21+1
        else:
            n11 = n11+1   
    '''
    ita = np.nditer (y_pred_array)
    itb = np.nditer (y_val_array)
    while not ita.finished:
        if (ita[0]==2)&(itb[0]==2):
            n22 = n22+1
        elif (ita[0]==1)&(itb[0]==2):
            n12 = n12+1
        elif (ita[0]==2)&(itb[0]==1):
            n21 = n21+1
        else:
            n11 = n11+1
        ita.iternext()
        itb.iternext()
    '''
    try:       
        Precall = n22 / ( n12 + n22)
        Pprecision = n22 / ( n21 + n22)
        f1_score = 2 / (1/Precall + 1/Pprecision)
        FP = n21/(n21 + n11)
        FN = n12/(n12 +n22 )
        BER = 1/2*(FP+FN)   
    except Exception as ex:
        print (ex.message)
    print ("n11:.."+str(n11)+"..n12:.."+str(n12)+"..n21:.."+str(n21)+"..n22:.."+str(n22))
    print ("TP:"+str(Precall))
    print ("f1 score:" + str(f1_score))
    print ("FP:"+ str(FP))
    print ("BER:" + str(BER))
    return Precall,f1_score,BER,FP

In [ ]:
#algorithm : {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}
#weights ‘uniform’ ‘distance’ 
#n_neighbors : int, optional (default = 5)
#p  euclidean_distance (l2) for p = 2

In [12]:
neigh = KNeighborsClassifier(n_neighbors=15,algorithm='kd_tree',weights ='distance')
neigh.fit(X_train, y_train['label']) 

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=15, p=2,
           weights='distance')

In [13]:
y_pred = neigh.predict(X_val)

In [14]:
y_pred = {"label_pred":y_pred}
y_pred = pd.DataFrame(data=y_pred)

In [15]:
Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 

n11:..56320..n12:..5365..n21:..4887..n22:..3455
TP:0.39172335600907027
f1 score:0.4026337256729985
FP:0.07984380871468949
BER:0.34406022635280964


In [66]:
#double the data of class2
start=time.time()
for algo in ['auto', 'ball_tree', 'kd_tree']: 
    for weight in ['distance','uniform']:
        for k_value in [5,10,20,40,60,80]:
            print ("current algorithm: " + algo +", weight:"+ weight+", numberofneighbors:"+str(k_value))
            neigh = KNeighborsClassifier(n_neighbors=k_value,algorithm=algo,weights =weight)
            neigh.fit(train_double_feature, train_double_label['label']) 
            y_pred = neigh.predict(X_val)
            y_pred = {"label_pred":y_pred}
            y_pred = pd.DataFrame(data=y_pred)
            Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 
print("fit time:%5.1fminute"%((time.time()-start)/60))

current algorithm: auto, weight:distance, numberofneighbors:5
n11:..51511..n12:..4369..n21:..9696..n22:..4451
TP:0.5046485260770975
f1 score:0.38759959942526234
FP:0.1584132533860506
BER:0.32688236365447654
current algorithm: auto, weight:distance, numberofneighbors:10
n11:..51945..n12:..4211..n21:..9262..n22:..4609
TP:0.522562358276644
f1 score:0.40624035961394384
FP:0.1513225611449671
BER:0.31438010143416156
current algorithm: auto, weight:distance, numberofneighbors:20
n11:..52540..n12:..4201..n21:..8667..n22:..4619
TP:0.5236961451247165
f1 score:0.417895593956392
FP:0.14160145081444933
BER:0.3089526528448664
current algorithm: auto, weight:distance, numberofneighbors:40
n11:..53026..n12:..4276..n21:..8181..n22:..4544
TP:0.5151927437641723
f1 score:0.42181480621954054
FP:0.1336611825444802
BER:0.30923421939015394
current algorithm: auto, weight:distance, numberofneighbors:60
n11:..53255..n12:..4287..n21:..7952..n22:..4533
TP:0.5139455782312925
f1 score:0.4255339122271767
FP:0.129919

In [60]:
#class1 choose 80000 ,class2 choose 40000
start=time.time()
for algo in ['auto', 'ball_tree', 'kd_tree']: 
    for weight in ['distance','uniform']:
        for k_value in [5,10,20,40,60,80]:
            print ("current algorithm: " + algo +", weight:"+ weight+", numberofneighbors:"+str(k_value))
            neigh = KNeighborsClassifier(n_neighbors=k_value,algorithm=algo,weights =weight)
            neigh.fit(sample_feature, sample_label['label']) 
            y_pred = neigh.predict(X_val)
            y_pred = {"label_pred":y_pred}
            y_pred = pd.DataFrame(data=y_pred)
            Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 
print("fit time:%5.1fminute"%((time.time()-start)/60))

current algorithm: auto, weight:distance, numberofneighbors:5
n11:..51187..n12:..4188..n21:..10020..n22:..4632
TP:0.5251700680272109
f1 score:0.39468302658486704
FP:0.16370676556603003
BER:0.31926834876940957
current algorithm: auto, weight:distance, numberofneighbors:10
n11:..52228..n12:..4200..n21:..8979..n22:..4620
TP:0.5238095238095238
f1 score:0.4121504081359561
FP:0.14669890698776283
BER:0.31144469158911947
current algorithm: auto, weight:distance, numberofneighbors:20
n11:..52940..n12:..4265..n21:..8267..n22:..4555
TP:0.5164399092970522
f1 score:0.42094076333056096
FP:0.13506625059225252
BER:0.3093131706476002
current algorithm: auto, weight:distance, numberofneighbors:40
n11:..53365..n12:..4259..n21:..7842..n22:..4561
TP:0.5171201814058957
f1 score:0.4298167082881779
FP:0.1281226003561684
BER:0.30550120947513637
current algorithm: auto, weight:distance, numberofneighbors:60
n11:..53543..n12:..4137..n21:..7664..n22:..4683
TP:0.530952380952381
f1 score:0.4424812207681769
FP:0.125

In [58]:
#class1 choose 40000 ,class2 choose 30000
start=time.time()
for algo in ['auto', 'ball_tree', 'kd_tree']: 
    for weight in ['distance','uniform']:
        for k_value in [5,10,20,40,60,80]:
            print ("current algorithm: " + algo +", weight:"+ weight+", numberofneighbors:"+str(k_value))
            neigh = KNeighborsClassifier(n_neighbors=k_value,algorithm=algo,weights =weight)
            neigh.fit(sample_feature, sample_label['label']) 
            y_pred = neigh.predict(X_val)
            y_pred = {"label_pred":y_pred}
            y_pred = pd.DataFrame(data=y_pred)
            Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 
print("fit time:%5.1fminute"%((time.time()-start)/60))

current algorithm: auto, weight:distance, numberofneighbors:5
n11:..47812..n12:..3498..n21:..13395..n22:..5322
TP:0.6034013605442177
f1 score:0.3865344808802702
FP:0.2188475174408156
BER:0.30772307844829894
current algorithm: auto, weight:distance, numberofneighbors:10
n11:..48965..n12:..3403..n21:..12242..n22:..5417
TP:0.614172335600907
f1 score:0.40915442426073495
FP:0.2000098028003333
BER:0.29291873359971315
current algorithm: auto, weight:distance, numberofneighbors:20
n11:..49744..n12:..3199..n21:..11463..n22:..5621
TP:0.6373015873015873
f1 score:0.4339870290302656
FP:0.18728250036760502
BER:0.27499045653300885
current algorithm: auto, weight:distance, numberofneighbors:40
n11:..50192..n12:..3123..n21:..11015..n22:..5697
TP:0.6459183673469387
f1 score:0.4462635124549585
FP:0.1799630761187446
BER:0.26702235438590294
current algorithm: auto, weight:distance, numberofneighbors:60
n11:..50373..n12:..3074..n21:..10834..n22:..5746
TP:0.6514739229024943
f1 score:0.45244094488188974
FP:0.

In [10]:
#the original dataset
start=time.time()
for algo in ['auto', 'ball_tree', 'kd_tree','brute']: 
    for weight in ['distance','uniform']:
        for k_value in [5,10,20,40,60,80]:
            print ("current algorithm: " + algo +", weight:"+ weight+", numberofneighbors:"+str(k_value))
            neigh = KNeighborsClassifier(n_neighbors=k_value,algorithm=algo,weights =weight)
            neigh.fit(X_train, y_train['label']) 
            y_pred = neigh.predict(X_val)
            y_pred = {"label_pred":y_pred}
            y_pred = pd.DataFrame(data=y_pred)
            Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 
print("fit time:%5.1fminute"%((time.time()-start)/60))

current algorithm: auto, weight:distance, numberofneighbors:5
n11:..55302..n12:..5215..n21:..5905..n22:..3605
TP:0.4087301587301587
f1 score:0.39334424440807414
FP:0.09647589328018037
BER:0.3438728672750108
current algorithm: auto, weight:distance, numberofneighbors:10
n11:..56070..n12:..5274..n21:..5137..n22:..3546
TP:0.4020408163265306
f1 score:0.40518768211163797
FP:0.0839283088535625
BER:0.3409437462635159
current algorithm: auto, weight:distance, numberofneighbors:20
n11:..56456..n12:..5415..n21:..4751..n22:..3405
TP:0.38605442176870747
f1 score:0.4011545711592837
FP:0.07762184063914258
BER:0.3457837094352175
current algorithm: auto, weight:distance, numberofneighbors:40
n11:..56710..n12:..5535..n21:..4497..n22:..3285
TP:0.37244897959183676
f1 score:0.3957354535598121
FP:0.0734719884980476
BER:0.3505115044531054
current algorithm: auto, weight:distance, numberofneighbors:60
n11:..56819..n12:..5572..n21:..4388..n22:..3248
TP:0.3682539682539683
f1 score:0.39474963539134666
FP:0.0716

MemoryError: 

In [93]:
start=time.time()
for algo in ['auto', 'ball_tree', 'kd_tree']: 
    for weight in ['distance','uniform']:
        for k_value in [5,6,8,10,12,14,16,18,20]:
            print ("current algorithm: " + algo +", weight:"+ weight+", numberofneighbors:"+str(k_value))
            neigh = KNeighborsClassifier(n_neighbors=k_value,algorithm=algo,weights =weight)
            neigh.fit(X_train, y_train['label']) 
            y_pred = neigh.predict(X_val)
            y_pred = {"label_pred":y_pred}
            y_pred = pd.DataFrame(data=y_pred)
            Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 
print("fit time:%5.1fminute"%((time.time()-start)/60))

current algorithm: auto, weight:distance, numberofneighbors:5
n11:..55551..n12:..5311..n21:..5656..n22:..3509
TP:0.3978458049886621
f1 score:0.3902140672782875
FP:0.09240773114186286
BER:0.34728096307660034
current algorithm: auto, weight:distance, numberofneighbors:6
n11:..55615..n12:..5291..n21:..5592..n22:..3529
TP:0.40011337868480723
f1 score:0.3934005908254835
FP:0.09136209910631136
BER:0.34562436021075205
current algorithm: auto, weight:distance, numberofneighbors:8
n11:..56070..n12:..5387..n21:..5137..n22:..3433
TP:0.38922902494331063
f1 score:0.39482461184588846
FP:0.0839283088535625
BER:0.3473496419551259
current algorithm: auto, weight:distance, numberofneighbors:10
n11:..56408..n12:..5443..n21:..4799..n22:..3377
TP:0.3828798185941043
f1 score:0.39738762061661564
FP:0.0784060646658062
BER:0.34776312303585094
current algorithm: auto, weight:distance, numberofneighbors:12
n11:..56563..n12:..5512..n21:..4644..n22:..3308
TP:0.3750566893424036
f1 score:0.39446696875745285
FP:0.075

n11:..56408..n12:..5443..n21:..4799..n22:..3377
TP:0.3828798185941043
f1 score:0.39738762061661564
FP:0.0784060646658062
BER:0.34776312303585094
current algorithm: kd_tree, weight:distance, numberofneighbors:12
n11:..56563..n12:..5512..n21:..4644..n22:..3308
TP:0.3750566893424036
f1 score:0.39446696875745285
FP:0.07587367457970494
BER:0.35040849261865065
current algorithm: kd_tree, weight:distance, numberofneighbors:14
n11:..56707..n12:..5583..n21:..4500..n22:..3237
TP:0.36700680272108843
f1 score:0.391012864649393
FP:0.07352100249971409
BER:0.3532570998893128
current algorithm: kd_tree, weight:distance, numberofneighbors:16
n11:..56809..n12:..5614..n21:..4398..n22:..3206
TP:0.3634920634920635
f1 score:0.3904042864101315
FP:0.0718545264430539
BER:0.3541812314754952
current algorithm: kd_tree, weight:distance, numberofneighbors:18
n11:..56886..n12:..5642..n21:..4321..n22:..3178
TP:0.3603174603174603
f1 score:0.3894846497947178
FP:0.07059650040028101
BER:0.3551395200414103
current algori